In [ ]:
#Basics
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import sys
import os
import csv
import time
import random
import pandas as pd

In [ ]:
val_file = pd.read_csv('/home/kvattis/Documents/data/train_dataset_control_AT_Mel_Spec_2022_noise_red2_severity_v9.csv', names=["P_ID", "Address", "Label","Date"])
val_file['Study'] = val_file['Address'].apply(lambda x: 'Audio_all' if x[0] == 'A' else ('Biogen_SCA' if x[0] == 'B' else 'Remote_AT') )

In [ ]:
#Read the Patient info csv and extract the diagnosis
Patient_info = pd.read_csv('/home/kvattis/Documents/data/Audio_All/2021_01_27_All_Tasks-data.csv',encoding='latin1')
Patient_info['gen_diagnosis'] = Patient_info['gen_diagnosis'].replace(['AT'], 'Ataxia')
Patient_info['P_ID'] = Patient_info['ID'].apply(lambda x: x[0:5] )
Patient_info.set_index('P_ID', inplace = True)

In [ ]:
Patient_info.columns

In [ ]:
Patient_info

In [ ]:
val_file

In [ ]:
val_file['P_ID'].unique()

In [ ]:
with open('/home/kvattis/Documents/speech_analysis/Model_training_map/train_severity_v9.npy', 'wb') as f:
    np.save(f,val_file['P_ID'].unique())

In [ ]:
val_file[val_file['Study'] == 'Remote_AT']

val_file.set_index('P_ID', inplace = True)

val_file.loc[30006]

pd.options.display.max_colwidth = 100


val_file['P_ID'].unique()

In [ ]:
val_file =val_file.to_numpy()

In [ ]:
Patient_info_biogen = pd.read_csv('/home/kvattis/Documents/data/Biogen_SCA_Speech_Data/Subjects_BiogenSCA_version2.csv',encoding='latin1')
Patient_info_biogen.set_index('subject_id', inplace = True)


In [ ]:
Patient_info_biogen

In [ ]:
Patient_info_biogen['bars_total_excl_miss_1'].loc[30006]

In [ ]:
Patient_info_biogen['age'].loc[30011] 

Patient_info_remote = pd.read_csv('/home/kvattis/Documents/data/Remote_AT_Speech_Data/Subjects_Remote_Behavior.csv',encoding='latin1')
Patient_info_remote.set_index('subject_id', inplace = True)

In [ ]:
Patient_info_remote = pd.read_csv('/home/kvattis/Documents/data/Remote_AT_Speech_Data/A-T-Table.csv',encoding='latin1')
Patient_info_remote.set_index('subject_id', inplace = True)

In [ ]:
Patient_info_remote.columns

Patient_info_remote[['neuropheno_id','Neuropheno_session_for_BARS', 'Age']]

In [ ]:
Patient_info_remote[['Neuropheno Study Link','Neuropheno_session_for_BARS', 'Age_2021']]

In [ ]:
info = []
bars_list = []
i = 0
for xx in val_file:
    if xx[4] == 'Audio_all':
        old_id = xx[1][-26:-10]
        bars_info = Patient_info[['bars_total','ID', 'bars_arm_L', 'bars_arm_R', 'bars_leg_L', 'bars_leg_R', 'bars_oculomotor', 'bars_gait']].loc[str(xx[0])]
        if bars_info.shape == (8,):
            bars = bars_info[0]
            bars_arm = bars_info[2] +  bars_info[3]
            bars_leg = bars_info[4] +  bars_info[5] 
            bars_occ = bars_info[6]
            bars_gait = bars_info[7]
            bars_speech = bars - (bars_arm + bars_leg + bars_occ + bars_gait)
        else:
            bars = bars_info[bars_info['ID'] == old_id]['bars_total'][0]
            bars_arm = bars_info[bars_info['ID'] == old_id]['bars_arm_L'][0] + bars_info[bars_info['ID'] == old_id]['bars_arm_R'][0] 
            bars_leg = bars_info[bars_info['ID'] == old_id]['bars_leg_L'][0] + bars_info[bars_info['ID'] == old_id]['bars_leg_R'][0] 
            bars_occ = bars_info[bars_info['ID'] == old_id]['bars_oculomotor'][0]
            bars_gait = bars_info[bars_info['ID'] == old_id]['bars_gait'][0]
            bars_speech = bars - (bars_arm + bars_leg + bars_occ + bars_gait)
            
    
        sex_ = Patient_info['sex'].loc[str(xx[0])]
        if len(sex_) == 1:
            sex = sex_[0]
        else:
            sex = sex_.unique()[0] 
            
        age_info = Patient_info[['age','ID']].loc[str(xx[0])]
        if age_info.shape == (2,):
            age = age_info[0]
        else:
            age = age_info[age_info['ID'] == old_id]['age'][0]
        info.append([i,str(xx[0]),sex,bars,age, bars_speech, str(xx[0]) + str(xx[3])])
        
    elif xx[4] == 'Biogen_SCA':
        
        if xx[0]<30000:
            id_ = Patient_info_biogen[Patient_info_biogen['neuropheno_id']==float(xx[0])].index.to_list()[0]
        else:
            id_ = xx[0]
        
        bars_1 = Patient_info_biogen['bars_total_excl_miss_1'].loc[id_]
        bars_2 = Patient_info_biogen['bars_total_excl_miss_2'].loc[id_]
        bars = (bars_1 + bars_2) / 2.
        
        bars_speech_1 = Patient_info_biogen['bars_speech_1'].loc[id_]
        bars_speech_2 = Patient_info_biogen['bars_speech_2'].loc[id_]
        bars_speech = (bars_speech_1 + bars_speech_2) / 2.
        
        sex_ = Patient_info_biogen['sex'].loc[id_]
        sex = 'F' if sex_ == 1 else 'M'
        age = Patient_info_biogen['age'].loc[id_]
        info.append([i,str(xx[0]),sex,bars,age, bars_speech/10., str(xx[0]) + str(xx[3])])
        
    else:
        
        if xx[0]<20000:
            id_ = Patient_info_remote[Patient_info_remote['Neuropheno Study Link']==float(xx[0])].index.to_list()[0]
        else:
            id_ = xx[0]
                
        neuro_ph_session = Patient_info_remote['Neuropheno_session_for_BARS'].loc[id_]

        if pd.isnull(neuro_ph_session):
            bars = np.nan
            bars_speech = np.nan
        else:
            bars_info = Patient_info[Patient_info['ID'] == neuro_ph_session]
            bars = bars_info['bars_total'][0]
            bars_speech = bars_info['bars_total'][0] - (bars_info['bars_arm_L'][0] + bars_info['bars_arm_R'][0] + bars_info['bars_leg_L'][0] + bars_info['bars_leg_R'][0] + bars_info['bars_oculomotor'][0] + bars_info['bars_gait'][0])
        sex = Patient_info_remote['Sex'].loc[id_]
        age = Patient_info_remote['Age'].loc[id_]
        info.append([i,str(xx[0]),sex,bars,age, bars_speech, str(xx[0]) + str(xx[3])])
    
    bars_list.append(bars)
    i+=1

print(len(info))

In [ ]:
info

In [ ]:
bars_list

In [ ]:
plt.hist(bars_list, density=True, bins=[x+1.5 for x in np.arange(-1.5,30,3)])  # density=False would make counts
plt.ylabel('Probability')
plt.xlabel('Data');

In [ ]:
hist_val = plt.hist(bars_list, density=False, bins=[x+1.5 for x in np.arange(-1.5,30,5) ])  # density=False would make counts

In [ ]:
hist_val[0]

with open('/home/kvattis/Documents/data/val_demo_Mel_cnn_nr2_latest_v4.csv',"w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(info)